In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import torch
import torchvision
from utils import plot_image
import torch.nn.functional as F

### Class labels
airplane: 0 \
automobile: 1\
bird: 2\
cat: 3\
deer: 4\
dog: 5\
frog: 6\
horse: 7\
ship: 8\
truck: 9

In [3]:
ind_to_classname = {0:'airplane',
                    1:'automobile',
                    2:'bird',
                    3:'cat',
                    4:'deer',
                    5:'dog',
                    6:'frog',
                    7:'horse',
                    8:'ship',
                    9:'truck',
                   }

In [5]:
adv = np.load("adversarials/fgsm.npy")

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

real_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)

real_loader = torch.utils.data.DataLoader(
    real_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified


In [ ]:
random_index = np.random.randint(0, min(len(adv), len(real_dataset)))
random_real = torch.argmax(net(real_dataset[random_index][0].unsqueeze(0).to(device))).item()
print(f"network prediction: {ind_to_classname[random_real]}, real class: {ind_to_classname[real_dataset.targets[random_index]]}")
plot_image(real_dataset[random_index], title="real")

In [ ]:
random_adv = torch.argmax(net(torch.from_numpy(adv[random_index]).unsqueeze(0).to(device))).item()
print(f"network prediction on adversarial: {ind_to_classname[random_adv]}")
plot_image(adv[random_index], title='adversarial')

In [99]:
from vgg import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

VGG_16 = [64, 64, 'pooling', 128, 128, 'pooling', 256, 256,
          256, 'pooling', 512, 512, 512, 'pooling', 512, 512, 512, 'pooling']

model = VGG(VGG_16)
model = net.to(device)
model.eval()

model.load_state_dict(torch.load('models_weights/vggModel'))

<All keys matched successfully>

In [128]:
F.softmax(net(torch.from_numpy(adv[3:5]).to(device)))

<ipython-input-128-0130cff5a935>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(net(torch.from_numpy(adv[3:5]).to(device)))


tensor([[3.5990e-01, 4.4700e-07, 5.9962e-01, 3.8904e-02, 1.2582e-03, 4.4070e-05,
         7.0203e-05, 1.4304e-05, 1.6763e-04, 2.6480e-05],
        [3.5204e-06, 3.0380e-07, 5.7217e-01, 3.8227e-06, 2.6843e-02, 2.2845e-06,
         4.0097e-01, 1.0228e-06, 6.1309e-06, 3.8422e-09]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

In [ ]:
class ml_adv_detector():
    
    def __init__(self, network, detector, params=None):
        '''
        network - neural network that is used to predict softmax of natural
        and adversarial data
        
        detector - ML model that will distinguish will be trained on softmax
        distributions
        
        params: expected to be a dictionary of detector parameters
        '''
        self.network = network
        self.detector = detector
        if params != None:
            self.detector.set_params(params)
        
    def fit(self, X_real, X_adversarial):
        '''   
        X_real - natural data
        
        X_adversarial - adversarial data
        '''
        X = np.concatenate((model.predict(X_real), model.predict(X_adversarial)), axis=0)
        y = np.concatenate((np.array([0] * X_real.shape[0]),
                            np.array([1] * X_adversarial.shape[0])))
        X, y = shuffle(X, y)
        
        self.detector.fit(X, y)                    
            
    def predict(self, X):
        '''
        return result of detection (1 - adversarial, 0 - natural)
        X - data to be detected
        '''
        X = self.network.predict(X)
        return np.round(self.detector.predict(X))